In [1]:
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import math
import glob
from pymsfilereader import MSFileReader

from PyQt5.QtCore import *
from PyQt5.QtGui import *
from PyQt5.QtWidgets import *
import sys
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
import re

In [2]:
class FileBrowser(QWidget):
    ImportFile = 0
    SaveImage = 1
    
    def __init__(self, title, mode=ImportFile):
        QWidget.__init__(self)
        layout = QHBoxLayout()
        self.setLayout(layout)
        self.browser_mode = mode
        self.dirpath = QDir.currentPath()
        self.label = QLabel("Import a .raw file:")
        self.label.setText(title)
        self.label.setFixedWidth(65)
        self.label.setFont(QFont("Arial",weight=QFont.Bold))
        self.label.setAlignment(Qt.AlignLeft | Qt.AlignVCenter)
        layout.addWidget(self.label)
        self.lineEdit = QLineEdit(self)
        self.lineEdit.setFixedWidth(180)
        layout.addWidget(self.lineEdit)
        self.button = QPushButton('Search')
        self.button.clicked.connect(self.getFile)
        layout.addWidget(self.button)
        layout.addStretch()
        
    def setMode(mode):
        self.mode = mode
    
    def getFile(self):
        self.filepaths = []
        if self.browser_mode == FileBrowser.ImportFile:
            self.filepaths.append(QFileDialog.getOpenFileName(self, caption='Choose File',
                                                    directory=self.dirpath)[0])
        else:
            options = QFileDialog.Options()
            if sys.platform == 'darwin':
                options |= QFileDialog.DontUseNativeDialog
            self.filepaths.append(QFileDialog.getExistingDirectory(self, caption='Save/Save As',
                                                    directory=self.dirpath))
        if len(self.filepaths) == 0:
            return
        elif len(self.filepaths) == 1:
            self.lineEdit.setText(self.filepaths[0])
        else:
            self.lineEdit.setText(",".join(self.filepaths))
    
    def setLabelWidth(self, width):
        self.label.setFixedWidth(width)   
        
    def setlineEditWidth(self, width):
        self.lineEdit.setFixedWidth(width)
        
    def getPaths(self):
        return self.filepaths
    
    
class Window(QDialog):
    # constructor
    def __init__(self, parent=None):
        QDialog.__init__(self, parent)
        # Ensure our window stays in front and give it a title
        self.setWindowFlags(Qt.WindowStaysOnTopHint)
        self.setWindowTitle("File Browsing Dialog")
        #self.setFixedSize(400, 300)
        self.setGeometry(400, 400, 500, 500)
        self.figure = plt.figure()
        self.canvas = FigureCanvas(self.figure)
        # Create and assign the main (vertical) layout.
        vlayout = QVBoxLayout()
        self.setLayout(vlayout)
        self.fileBrowserPanel(vlayout)
        vlayout.addStretch()
        self.imagePanel(vlayout)
        vlayout.addStretch()
        self.addButtonPanel(vlayout)
        self.show()
        
    def fileBrowserPanel(self, parentLayout):
        flayout = QVBoxLayout()
        self.fileFB = FileBrowser('Import File', FileBrowser.ImportFile)
        self.saveFB = FileBrowser('Save Image', FileBrowser.SaveImage)
        flayout.addWidget(self.fileFB)
        flayout.addWidget(self.saveFB)
        flayout.addStretch()
        parentLayout.addLayout(flayout)
        
    def imagePanel(self, parentLayout):
        imlayout = QVBoxLayout()
        imlayout.addWidget(self.canvas)
        parentLayout.addLayout(imlayout)
          
    def addButtonPanel(self, parentLayout):
        hlayout = QHBoxLayout()
        hlayout.addStretch()
        self.closeButton = QPushButton("Close")
        self.closeButton.clicked.connect(self.closeButtonAction)
        hlayout.addWidget(self.closeButton)
        parentLayout.addLayout(hlayout)
        self.button = QPushButton("Run")
        self.button.clicked.connect(self.buttonAction)
        hlayout.addWidget(self.button)
        parentLayout.addLayout(hlayout)

    def process_one_file(self, file):
        frame = []
        try:
            rawfile=MSFileReader(file)
            for i in range(rawfile.FirstSpectrumNumber, rawfile.LastSpectrumNumber + 1):
                labels, flags = rawfile.GetLabelData(i)
                cur_df = pd.DataFrame({'x': i , 'm/z': labels.mass, 'intensity': labels.intensity, 'resolution': labels.resolution, 'noise': labels.noise})
                frame.append(cur_df)
            return frame
        except IOError:
            print("file error {file}".format(file=file))
            
    def file_to_csv(self, file):
        to_csv=file[0]
        for i in range(1,len(file)):
            cat = file[i]
            to_csv = pd.concat([to_csv, cat])
        to_csv['S/N'] = to_csv['intensity'] / to_csv['noise']
        to_csv = to_csv.sort_values(by = ['S/N'], ascending = False).reset_index(drop = True).iloc[:1001]
        return to_csv
        
    def buttonAction(self, parentLayout):
        print(self.fileFB.getPaths())
        demoFile = self.fileFB.getPaths()[0]
        self.file_to_csv(self.process_one_file(file=demoFile)).to_csv('demoFile_to_csv.csv')
        df = pd.read_csv("demoFile_to_csv.csv")
        x = df["m/z"]
        y = df["resolution"]
        x_log = []
        x_log_sq = []
        y_log = []
        for i in range(len(y)):
            x_log.append(math.log(x[i]))
            x_log_sq.append( (math.log(x[i]))**2 )
            y_log.append(math.log(y[i]))
        lnXlnY = []
        for i in range(len(y)):
            lnXlnY.append( x_log[i] * y_log[i] )
        sum_lnXlnY = np.sum(lnXlnY)
        sum_lnX = np.sum(x_log)
        sum_lnY = np.sum(y_log)
        sum_lnX2 = np.sum(x_log_sq)
        B = (len(y) * sum_lnXlnY - sum_lnX * sum_lnY) / (len(y) * sum_lnX2 - sum_lnX ** 2)
        A = math.exp( (sum_lnY - B * sum_lnX) / len(y) )
        # ln(y) = ln(A) + B*ln(x)
        lnY_estimate = []
        for i in range(len(y)):
            lnY_estimate.append( math.log(A) + B * x_log[i] )

        mean = np.mean(y_log)
        residuals = []
        tot = []
        for i in range(len(y)):
            residuals.append( (y_log[i] - lnY_estimate[i])**2 )
            tot.append( (y_log[i] - mean)**2 )

        ss_res = np.sum(residuals)
        ss_tot = np.sum(tot)
        r_squared = 1 - (ss_res / ss_tot)
        r_squared, round(r_squared, 4)

        popt, pcov = curve_fit(lambda fx,k,t: k*fx**-t, x, y)
        x_linspace = np.linspace(min(x), max(x), 500)
        power_y = popt[0]*x_linspace**-popt[1]

        self.figure.clear()
        plt.close()
        plt.rcParams['figure.figsize'] = [5, 5]
        plt.rcParams["figure.dpi"] = 600
        #create a subplot
        ax = self.figure.add_subplot(111)

        scatter = ax.scatter(x, y, s=df["S/N"]/100, label='detected ions (size: S/N ratio)', c="blue", alpha=0.75, lw=0)
        plt.xlabel("m/z", fontproperties='Arial', fontsize=12)
        plt.ylabel("Resolution", fontproperties='Arial', fontsize=12)
        plt.xlim(0,1600)
        plt.ylim(20000,180000)
        plt.plot(x_linspace, power_y, label='regression curve', color = "red")
        print("result from scipy is not precise: A != " + str(popt[0]) + ", B != -" + str(popt[1]) 
              + "so I calculate the coefficient A value & index B value by myself," 
              + " giving out the same values as the values calculated by Excel, accompanying with the R-square value.")

        ax.annotate('y = ' + str(round(A)) + ' x * exp(' + str(round(B, 4)) + ')', (1000, 120000), horizontalalignment='center',
             verticalalignment='center', fontproperties='Arial', fontsize=10)
        ax.annotate('R-square = ' + str(round(r_squared, 4)), (1000, 110000), horizontalalignment='center',
             verticalalignment='center', fontproperties='Arial', fontsize=10)

        handles, labels = scatter.legend_elements(prop="sizes", alpha=0.6, color="lightgrey", num=4)
        legend = ax.legend(handles, labels, loc="upper right", title="S/N ratio (*100)", title_fontsize=8.5, fontsize=8.5)
        #draw new graph
        self.canvas.draw()
        
        savePath = str(self.saveFB.getPaths()[0]) + "/regression_demo.jpg"
        plt.savefig(savePath, bbox_inches='tight')
        plt.close()
        
    def closeButtonAction(self):
        sys.exit(app.exec_())

In [ ]:
if __name__ == '__main__':
    # Create the Qt Application
    app = QApplication(sys.argv)
    demo = Window() # <<-- Create an instance
    demo.show()
    sys.exit(app.exec_())

['C:/Users/resea/Python_LAB/Work/GUI/GUI/demo_raw/DESI_MSI/KidneyHomo_Test_20220628_18.raw']
result from scipy is not precise: A != 938737.9729803527, B != -0.5073526890489576so I calculate the coefficient A value & index B value by myself, giving out the same values as the values calculated by Excel, accompanying with the R-square value.
